In [1]:
from dotenv import load_dotenv
from kubernetes import client
from openshift.dynamic import DynamicClient
from openshift.helper.userpassauth import OCPLoginConfiguration
import os
import pandas as pd
import requests
import urllib3

load_dotenv() 

# disable pesky urllib3 ssl warnings.
urllib3.disable_warnings()

OCP_APIHOST = os.environ["OCP_APIHOST"]
OCP_USERNAME = os.environ["OCP_USERNAME"]
OCP_PASSWORD = os.environ["OCP_PASSWORD"]
QPC_BASE_URL = os.environ["QPC_BASE_URL"]
QPC_USERNAME = os.environ["QPC_USERNAME"]
QPC_PASSWORD = os.environ["QPC_PASSWORD"]
QPC_REPORT_ID = os.environ["QPC_REPORT_ID"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

kubeConfig = OCPLoginConfiguration(ocp_username=OCP_USERNAME, ocp_password=OCP_PASSWORD)
kubeConfig.host = OCP_APIHOST
kubeConfig.verify_ssl = False

kubeConfig.get_token()

 
k8s_client = client.ApiClient(kubeConfig)
 
dyn_client = DynamicClient(k8s_client)
# v1_projects = dyn_client.resources.get(api_version='project.openshift.io/v1', kind='Project')
# project_list = v1_projects.get()

In [2]:
routes_client = dyn_client.resources.get(api_version='route.openshift.io/v1', kind='Route')
routes = routes_client.get(namespace="openshift-monitoring", field_selector="metadata.name=prometheus-k8s").items
prometheus_host = routes[0]["spec"]["host"] + routes[0]["spec"].get("path", "/api")
prometheus_host

'prometheus-k8s-openshift-monitoring.apps.sharedocp4upi412ovn.lab.upshift.rdu2.redhat.com/api'

In [3]:
console_client = dyn_client.resources.get(
    api_version="config.openshift.io/v1", kind="Console"
)
console_url = console_client.get().items[0]["status"]["consoleURL"]
print(console_url)

# prometheus_query="node_role_os_version_machine:cpu_capacity_sockets:sum"
#prometheus_query="workload:capacity_physical_cpu_cores:sum" # https://github.com/openshift/cluster-monitoring-operator/blob/1c595854ab4acbefd70446b54ae314016f400d8e/jsonnet/rules.libsonnet#L249-L251
# prometheus_query="node_role_os_version_machine:cpu_capacity_cores:sum" # https://github.com/openshift/cluster-monitoring-operator/blob/f365468055303edc8a4bb93e494bcbdedcf587bd/manifests/0000_50_cluster-monitoring-operator_04-config.yaml#L248-L253
# slightly modified version of what cluster-monitoring-operator uses for getting cpu_capacity_sockets
prometheus_query = '''
count by(instance, label_node_hyperthread_enabled, label_node_openshift_io_os_id, label_node_role_kubernetes_io_master, label_node_role_kubernetes_io_infra) 
(max 
    by(node, instance, package, label_node_hyperthread_enabled, label_node_openshift_io_os_id, label_node_role_kubernetes_io_master, label_node_role_kubernetes_io_infra)
    (cluster:cpu_core_node_labels)
)
'''
# other interesting metrics
# - node_cpu_info
# - machine_cpu_sockets
# - machine_cpu_cores
# - machine_cpu_physical_cores
# - node_network_info (macaddresses)
# - cluster_infrastructure_provider

# r.status_code, r.json()

def prometheus_json_to_df(query, value_key):
    response = requests.get(
        f"https://{prometheus_host}/v1/query?query={query}",
        verify=False,
        headers=kubeConfig.api_key,
    )
    assert response.ok
    return pd.DataFrame(
        {value_key:d["value"][1], **d["metric"]}
        for d in response.json()["data"]["result"]
    )

prometheus_json_to_df(prometheus_query, "sockets")


https://console-openshift-console.apps.sharedocp4upi412ovn.lab.upshift.rdu2.redhat.com


,sockets,instance,label_node_hyperthread_enabled,label_node_openshift_io_os_id,label_node_role_kubernetes_io_master
0,4,master-0.sharedocp4upi412ovn.lab.upshift.rdu2....,false,rhcos,true
1,4,master-1.sharedocp4upi412ovn.lab.upshift.rdu2....,false,rhcos,true
2,4,master-2.sharedocp4upi412ovn.lab.upshift.rdu2....,false,rhcos,true
3,4,worker-0.sharedocp4upi412ovn.lab.upshift.rdu2....,false,rhcos,NaN
4,4,worker-1.sharedocp4upi412ovn.lab.upshift.rdu2....,false,rhcos,NaN
5,4,worker-2.sharedocp4upi412ovn.lab.upshift.rdu2....,false,rhcos,NaN


In [4]:
prometheus_json_to_df("node_cpu_info", "?")

,?,__name__,cachesize,container,core,cpu,endpoint,family,instance,job,microcode,model,model_name,namespace,package,pod,service,stepping,vendor
0,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,master-0.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-pj88z,node-exporter,2,AuthenticAMD
1,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,master-1.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-bvfx9,node-exporter,2,AuthenticAMD
2,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,master-2.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-9rnhx,node-exporter,2,AuthenticAMD
3,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,worker-0.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-nkd8c,node-exporter,2,AuthenticAMD
4,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,worker-1.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-9gljp,node-exporter,2,AuthenticAMD
5,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,worker-2.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-2g925,node-exporter,2,AuthenticAMD
6,1,node_cpu_info,512 KB,kube-rbac-proxy,0,1,https,23,master-0.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,1,node-exporter-pj88z,node-exporter,2,AuthenticAMD
7,1,node_cpu_info,512 KB,kube-rbac-proxy,0,1,https,23,master-1.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,1,node-exporter-bvfx9,node-exporter,2,AuthenticAMD
8,1,node_cpu_info,512 KB,kube-rbac-proxy,0,1,https,23,master-2.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,1,node-exporter-9rnhx,node-exporter,2,AuthenticAMD
9,1,node_cpu_info,512 KB,kube-rbac-proxy,0,1,https,23,worker-0.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,1,node-exporter-nkd8c,node-exporter,2,AuthenticAMD


In [5]:
prometheus_json_to_df("node_network_info", "?")

,?,__name__,address,broadcast,container,device,endpoint,instance,job,namespace,operstate,pod,service,duplex
0,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,master-0.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-pj88z,node-exporter,NaN
1,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,master-1.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-bvfx9,node-exporter,NaN
2,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,master-2.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-9rnhx,node-exporter,NaN
3,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,worker-0.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-nkd8c,node-exporter,NaN
4,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,worker-1.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-9gljp,node-exporter,NaN
5,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,worker-2.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-2g925,node-exporter,NaN
6,1,node_network_info,0a:f2:eb:9c:58:fa,ff:ff:ff:ff:ff:ff,kube-rbac-proxy,genev_sys_6081,https,worker-0.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-nkd8c,node-exporter,NaN
7,1,node_network_info,2e:4d:54:f6:20:3b,ff:ff:ff:ff:ff:ff,kube-rbac-proxy,ovs-system,https,master-2.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,down,node-exporter-9rnhx,node-exporter,NaN
8,1,node_network_info,42:bc:2b:12:79:0b,ff:ff:ff:ff:ff:ff,kube-rbac-proxy,genev_sys_6081,https,master-1.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-bvfx9,node-exporter,NaN
9,1,node_network_info,5a:c6:b1:b9:b1:08,ff:ff:ff:ff:ff:ff,kube-rbac-proxy,ovs-system,https,master-1.sharedocp4upi412ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,down,node-exporter-bvfx9,node-exporter,NaN


In [6]:
# https://prometheus.io/docs/prometheus/latest/querying/api/
r = requests.get(
    f"https://{prometheus_host}/v1/metadata",
    verify=False,
    headers=kubeConfig.api_key,
)
r.status_code, r.json()


(200,
 {'status': 'success',
  'data': {'aggregator_openapi_v2_regeneration_count': [{'type': 'counter',
     'help': '[ALPHA] Counter of OpenAPI v2 spec regeneration count broken down by causing APIService name and reason.',
     'unit': ''}],
   'aggregator_openapi_v2_regeneration_duration': [{'type': 'gauge',
     'help': '[ALPHA] Gauge of OpenAPI v2 spec regeneration duration in seconds.',
     'unit': ''}],
   'aggregator_unavailable_apiservice': [{'type': 'gauge',
     'help': '[ALPHA] Gauge of APIServices which are marked as unavailable broken down by APIService name.',
     'unit': ''}],
   'alertmanager_alerts': [{'type': 'gauge',
     'help': 'How many alerts by state.',
     'unit': ''}],
   'alertmanager_alerts_invalid_total': [{'type': 'counter',
     'help': 'The total number of received alerts that were invalid.',
     'unit': ''}],
   'alertmanager_alerts_received_total': [{'type': 'counter',
     'help': 'The total number of received alerts.',
     'unit': ''}],
   'al

In [7]:

r = requests.get(
    f"https://{prometheus_host}/v1/rules",
    verify=False,
    headers=kubeConfig.api_key,
)
r.status_code, r.json()


(200,
 {'status': 'success',
  'data': {'groups': [{'name': 'CloudCredentialOperator',
     'file': '/etc/prometheus/rules/prometheus-k8s-rulefiles-0/openshift-cloud-credential-operator-cloud-credential-operator-alerts-55b21d2b-48ae-4cf2-b534-848640372dbd.yaml',
     'rules': [{'state': 'inactive',
       'name': 'CloudCredentialOperatorTargetNamespaceMissing',
       'query': 'cco_credentials_requests_conditions{condition="MissingTargetNamespace"} > 0',
       'duration': 300,
       'labels': {'severity': 'warning'},
       'annotations': {'description': "At least one CredentialsRequest custom resource has specified in its .spec.secretRef.namespace field a namespace which does not presently exist. This means the Cloud Credential Operator in the openshift-cloud-credential-operator namespace cannot process the CredentialsRequest resource. Check the conditions of all CredentialsRequests with 'oc get credentialsrequest -A' to find any CredentialsRequest(s) with a .status.condition showin

In [8]:

r = requests.get(
    f"https://{prometheus_host}/v1/labels",
    verify=False,
    headers=kubeConfig.api_key,
)
r.status_code, r.json()


(200,
 {'status': 'success',
  'data': ['From',
   'Local',
   'Remote',
   'To',
   '__name__',
   'access_mode',
   'action',
   'addr',
   'address',
   'alertmanager',
   'alertname',
   'alertstate',
   'apiserver',
   'apiservice',
   'approval',
   'attempts',
   'backend',
   'bios_date',
   'bios_vendor',
   'bios_version',
   'boot_id',
   'bound',
   'branch',
   'bridge',
   'broadcast',
   'build',
   'buildDate',
   'build_date',
   'build_phase',
   'build_user',
   'buildconfig',
   'cachesize',
   'call',
   'cause',
   'channel',
   'chassis_vendor',
   'chassis_version',
   'check',
   'checkName',
   'cidr',
   'client_api_version',
   'clocksource',
   'cloud_type',
   'cluster_id',
   'cluster_ip',
   'cluster_version',
   'code',
   'code_path',
   'collector',
   'command',
   'compiler',
   'completion_mode',
   'component',
   'concurrency_policy',
   'condition',
   'config',
   'configmap',
   'connection_method',
   'container',
   'container_id',
   'conta

In [9]:
r = requests.get(
    f"https://{prometheus_host}/v1/label/system_uuid/values",
    verify=False,
    headers=kubeConfig.api_key,
)
r.json()

{'status': 'success',
 'data': ['47c12e10-c5d8-4688-a5f9-c89574e71465',
  '4db6cf3c-4598-435c-88dd-715887bc8b30',
  '6738e507-5fa3-4ed8-b7c2-b33097d5828c',
  '8d6d2ff7-7011-427a-bde9-4129b2ee1c3f',
  '9c9d50ad-f1d4-4597-b06f-876ab6747c76',
  'aab78d4a-1d24-4a9e-9c44-694ea6882ba5']}

In [10]:
r = requests.get(
    f"https://{prometheus_host}/v1/query?query=machine_cpu_cores&query=machine_cpu_sockets",
    verify=False,
    headers=kubeConfig.api_key,
)
r.json()

{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'machine_cpu_cores',
     'boot_id': '162a40c1-36a1-48a2-8546-bddd8c2b4dab',
     'endpoint': 'https-metrics',
     'instance': '10.0.94.152:10250',
     'job': 'kubelet',
     'machine_id': '47c12e10c5d84688a5f9c89574e71465',
     'metrics_path': '/metrics/cadvisor',
     'namespace': 'kube-system',
     'node': 'master-0.sharedocp4upi412ovn.lab.upshift.rdu2.redhat.com',
     'service': 'kubelet',
     'system_uuid': '47c12e10-c5d8-4688-a5f9-c89574e71465'},
    'value': [1685395595.153, '4']},
   {'metric': {'__name__': 'machine_cpu_cores',
     'boot_id': '1c55857c-a00e-4762-883a-3d829a7fb71f',
     'endpoint': 'https-metrics',
     'instance': '10.0.92.95:10250',
     'job': 'kubelet',
     'machine_id': '4db6cf3c4598435c88dd715887bc8b30',
     'metrics_path': '/metrics/cadvisor',
     'namespace': 'kube-system',
     'node': 'worker-0.sharedocp4upi412ovn.lab.upshift.rdu2.redhat.com',
     '

In [11]:
df = prometheus_json_to_df("cluster_operator_conditions", "?")
df

,?,__name__,condition,endpoint,instance,job,name,namespace,pod,reason,service
0,1,cluster_operator_conditions,Available,metrics,10.0.93.146:9099,cluster-version-operator,authentication,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,AsExpected,cluster-version-operator
1,1,cluster_operator_conditions,Available,metrics,10.0.93.146:9099,cluster-version-operator,baremetal,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,WaitingForProvisioningCR,cluster-version-operator
2,1,cluster_operator_conditions,Available,metrics,10.0.93.146:9099,cluster-version-operator,cloud-controller-manager,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,AsExpected,cluster-version-operator
3,1,cluster_operator_conditions,Available,metrics,10.0.93.146:9099,cluster-version-operator,cloud-credential,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,AsExpected,cluster-version-operator
4,1,cluster_operator_conditions,Available,metrics,10.0.93.146:9099,cluster-version-operator,cluster-autoscaler,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,AsExpected,cluster-version-operator
...,...,...,...,...,...,...,...,...,...,...,...
143,1,cluster_operator_conditions,Upgradeable,metrics,10.0.93.146:9099,cluster-version-operator,service-ca,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,AsExpected,cluster-version-operator
144,1,cluster_operator_conditions,Upgradeable,metrics,10.0.93.146:9099,cluster-version-operator,storage,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,AsExpected,cluster-version-operator
145,0,cluster_operator_conditions,Upgradeable,metrics,10.0.93.146:9099,cluster-version-operator,version,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,MultipleReasons,cluster-version-operator
146,0,cluster_operator_conditions,UpgradeableAdminAckRequired,metrics,10.0.93.146:9099,cluster-version-operator,version,openshift-cluster-version,cluster-version-operator-586dd8fdbf-tt8s2,AdminAckRequired,cluster-version-operator
